### Load Necessary Packages

In [1]:
from pymongo import MongoClient
import random
import pandas as pd
import numpy as np

### Load Data Generated by recipe_tagger.ipynb

In [2]:
recipes = pd.read_csv(r"Documents/cooking project/recipes_updated2.csv")

In [3]:
def make_list(x):
    new = x.split(",")
    new = [y.strip(' ') for y in new]
    return new

recipes['keywords'] =  recipes['keywords'].apply(lambda x: make_list(x))

### Connect to Atlas Cluster

In [4]:
client = MongoClient('mongodb://rjnagley:<password>@cluster0-shard-00-00-3llbe.mongodb.net:27017,cluster0-shard-00-01-3llbe.mongodb.net:27017,cluster0-shard-00-02-3llbe.mongodb.net:27017/test?ssl=true&replicaSet=Cluster0-shard-0&authSource=admin&retryWrites=true&w=majority')
db = client.test

### Create Collection Called 'Recipes' from CSV

In [5]:
url = list(recipes['link'])
meal = list(recipes['meal'])
tags = list(recipes['keywords'])
cooktime = list(recipes['time'])
servings = list(recipes['yield'])

In [7]:
for x in range(0, len(recipes)):
    recipe = {
        'url' : url[x],
        'meal' : meal[x],
        'tags' : tags[x],
        'cooktime': cooktime[x],
        'servings': servings[x]
    }
    result=db.recipes.insert_one(recipe)

### Define Functions to create menu based on input of meal types, dietary restrictions, and preferences

In [16]:
def find_meal(meal,diet_res, preferences, menu_urls):
    random_tag = random.choice(preferences)
    diet_res.append(random_tag)
    meal = db.recipes.find_one({"meal":meal,"tags":{"$all": diet_res}})
    del diet_res[-1]
    if meal is not None:
        url = meal['url']
        if url in menu_urls:
            meal = None
    return meal

def create_meal_type_list(breakfast_days,lunch_days, dinner_days, dessert_days):
    meal_types = []
    for i in breakfast_days:
        meal_types.append('breakfast')
    for i in lunch_days:
        meal_types.append('lunch')
    for i in dinner_days:
        meal_types.append('dinner')
    for i in dessert_days:
        meal_types.append('dessert')
    return meal_types

def url_getter(breakfast_days,lunch_days, dinner_days, dessert_days, diet_res, preferences):
    menu_urls = []
    meal_types = create_meal_type_list(breakfast_days,lunch_days, dinner_days, dessert_days)
    print(meal_types)
    for meal in meal_types:
        selection = None
        while selection is None:
            selection = find_meal(meal, diet_res, preferences, menu_urls)
            if selection is not None:
                url = selection['url']
                #if url in menu_urls:
                    #selection = None
        menu_urls.append(url)     
    return menu_urls

In [17]:
breakfast_days = []
lunch_days = ['wednesday']
dinner_days = ['tuesday','wednesday']
dessert_days = []
diet_res = ['easy']
preferences = ['chinese','salad','chicken','italian','stew','italian','quick','egg','oreo','chocolate','quiche']
menu_urls = url_getter(breakfast_days,lunch_days, dinner_days, dessert_days, diet_res, preferences)
menu_urls

['lunch', 'dinner', 'dinner']


['https://damndelicious.net/2020/03/03/creamy-chicken-piccata/',
 'https://damndelicious.net/2014/08/13/easy-beef-broccoli/',
 'http://www.myrecipes.com/recipe/loaded-potato-soup-0']